In [5]:
import mysql.connector
import re
import os
import glob
import shutil

In [18]:
host = 'localhost'
port = 3306
user = 'root'
password = 'humaninterface'
conn = mysql.connector.connect(
    host=host,
    port=port,
    user=user,
    password=password,
)
print(conn.is_connected())

cur = conn.cursor()
database_name = 'childfren_drawing_analysis'
cur.execute('DROP DATABASE IF EXISTS {};'.format(database_name))
cur.execute("CREATE DATABASE {};".format(database_name))
conn.close()


conn = mysql.connector.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database_name,
)
print(conn.is_connected())

cur = conn.cursor()
table = 'answers'
cur.execute('DROP TABLE IF EXISTS {};'.format(table))
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS {} (
    `image_id` int not null,
    `questionnaire_id` int not null,
    `value` int not null,
    PRIMARY KEY(image_id, questionnaire_id)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci
    '''.format(table)
)
conn.close()


conn = mysql.connector.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database_name,
)
print(conn.is_connected())

cur = conn.cursor()
table = 'images'
cur.execute('DROP TABLE IF EXISTS {};'.format(table))
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS {} (
    `id` int auto_increment primary key,
    `year` int not null,
    `grade` int not null,
    `class` int not null,
    `theme_id` int not null,
    `student_number` int not null,
    `answered` int not null
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci
    '''.format(table)
)
conn.close()


image_dir = '/Users/human/tanamoto/research/art/children-drawing-analysis/children-drawing/face'
image_save_dir = '/Users/human/tanamoto/research/art/questionnaire-app/server/static/images'
image_ext = '.jpg'
image_path_list = glob.glob(image_dir+'/*'+image_ext)
conn = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database_name,
    )
cur = conn.cursor()
for image_path in image_path_list:
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    year, _, grade, class_, theme_id, student_number = re.split('_|-', image_name)
    year = int(year)
    grade = int(grade)
    class_ = int(class_)
    theme_id = int(theme_id)
    student_number = int(student_number)
    statement = '''
        INSERT INTO images values (null, {year}, {grade}, {class_}, {theme_id}, {student_number}, {answered});
    '''.format(year=year, grade=grade, class_=class_, theme_id=theme_id, student_number=student_number, answered=0)
    cur.execute(statement)
    conn.commit()
    statement = '''
        SELECT id FROM images 
            WHERE year={year} AND grade={grade} AND class={class_} AND theme_id={theme_id} AND student_number={student_number};
    '''.format(year=year, grade=grade, class_=class_, theme_id=theme_id, student_number=student_number)
    cur.execute(statement)
    rows = cur.fetchall()
    image_id = rows[0][0]
    image_save_path = os.path.join(image_save_dir, str(image_id)+image_ext)
    shutil.copy(image_path, image_save_path)
conn.close()

True
True
True
